In [1]:
import mysql.connector
import pandas as pd
from collections import defaultdict

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            user='root',
            password='123456',
            database='ASTOCK'
        )
        return connection
    except mysql.connector.Error as e:
        print(f"连接MySQL失败: {e}")
        return None

In [2]:
def get_min_market_value(year, month) -> list:
    # 获取市值数据
    market_values = get_market_values(year, month)

    # 按市值排序，选择前N个标的
    top_n = 10
    sorted_symbols = sorted(market_values, key=lambda x: x["market_value"])[:top_n]
    return sorted_symbols

def get_market_values(year, month) -> list:
    connection = create_connection()
    if not connection:
        return []
    
    finance_report_months = ['12', '03', '06', '09']
    year_of_interest = year - 1 if month < 4 else year
    report_month_of_interest = str(year_of_interest) + "-" + finance_report_months[(month - 1) // 3]

    cursor = connection.cursor()
    sql = f'''SELECT 
            a.symbol, a.name, a.update_time, (a.total_shares * b.close_price) AS market_value
            FROM finance a
            JOIN (
                SELECT 
                    symbol,
                    datetime,
                    close_price,
                    ROW_NUMBER() OVER (
                        PARTITION BY symbol, YEAR(datetime), MONTH(datetime) 
                        ORDER BY datetime DESC
                    ) as rn
                FROM daily
                WHERE exchange = 'SZSE'
                    AND datetime like '{report_month_of_interest}-%'
            ) b 
            ON a.symbol = b.symbol 
                AND YEAR(a.update_time) = YEAR(b.datetime)
                AND MONTH(a.update_time) = MONTH(b.datetime)
                AND b.rn = 1
            WHERE a.market = '深市'
                AND a.update_time like '{report_month_of_interest}-%'
                AND a.symbol like '00%'
                AND a.name not like '%ST%'
                AND a.name not like '%退'
            ;'''
    # print(f"SQL查询语句: {sql}")
    cursor.execute(sql)
    rows = cursor.fetchall()
    cursor.close()
    connection.close()

    # 将查询结果转换为字典列表
    values = [{"symbol": row[0], "name": row[1], "market_value": row[3]} for row in rows]
    return values

def convert_list_to_df(list_data)->pd.DataFrame:
    results = defaultdict(list)
    for data in list_data:
        for key, value in data.__dict__.items():
            results[key].append(value)
    df = pd.DataFrame.from_dict(results).set_index('datetime')
    return df



In [3]:
from vnpy.trader.optimize import OptimizationSetting
from vnpy_ctastrategy.backtesting import BacktestingEngine
from datetime import datetime
import sys
import os
import copy

parent_dir = os.path.abspath(os.path.join(os.path.curdir, ".."))
sys.path.append(parent_dir)
from chart_tools import data
from strategies.monthly_min_market_value_strategy_base import MonthlyMinMarketValueStrategy
from tools.common import sum_specified_keep_others

goods_symbols = []

start_day = datetime(2024, 4, 1)
end_day = datetime(2025, 3, 31)
data.set_start_date(start_day)
data.set_end_date(end_day)

month_first_list = []
# current_date = start_day.replace(day=1)  # 确保从每月的1日开始
current_date = start_day

while current_date <= end_day:
    month_first_list.append(current_date)

    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1, day=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1, day=1)

print(month_first_list)

capital = 1000000
total_profits = list()

record_df_dicts = list()
record_candle_dicts = list()
record_engines = list()
integrated_df = pd.DataFrame()
dfs = list()
for i in range(len(month_first_list)):
    if i > 0:
        capital += total_profits[i - 1] / 10  # add profit
    record_df_dicts.append({})
    record_candle_dicts.append({})
    start = month_first_list[i]
    end = month_first_list[i + 1] if i + 1 < len(month_first_list) else end_day
    end = end.replace(month=end.month + 1, day=9) if end.month < 12 else end.replace(year=end.year+1 ,month=1, day=9) # for sell
    print(f"开始日期: {start}")
    
    min_market_symbols = get_min_market_value(start.year, start.month)

    # sort by market value
    symbols_candidates = min_market_symbols[:]
    month_profits = list() 
    month_profit = 0
    print(f"候选标的：{symbols_candidates}")

    for symbol in symbols_candidates:
        engine = BacktestingEngine()
        vt_symbol = symbol["symbol"] + ".SZSE"
        engine.set_parameters(
            vt_symbol=vt_symbol,
            interval="d",
            start=start,
            end=end,
            rate=0.3/10000,
            slippage=0.2,
            size=100,
            pricetick=0.2,
            capital=capital,
        )
        engine.add_strategy(MonthlyMinMarketValueStrategy, {"initial_capital": capital, "current_month": start.month})

        engine.load_data()
        engine.run_backtesting()
        df = engine.calculate_result()
        dfs.append(df)
        
        record_df_dicts[i][symbol["symbol"]] = df
        record_candle_dicts[i][symbol["symbol"]] = engine.history_data
        res = engine.calculate_statistics(output=False)
        engine.cross_stop_order
        print(f"策略回测成功：{symbol['symbol']}，总净利润：{res['total_net_pnl']}")
        month_profits.append(res["total_net_pnl"])
        record_engines.append(copy.deepcopy(engine))
        
        data.add_tech_data(vt_symbol, 'daily_df', df)
        data.add_trade_data(vt_symbol, convert_list_to_df(engine.get_all_trades()))
        if res["total_net_pnl"] > 0:
            goods_symbols.append({"symbol": symbol["symbol"], "total_net_pnl": res["total_net_pnl"]})
    month_profit = sum(month_profits)
    print(f"{start.year}-{start.month}月总净利润：{month_profit}")
    total_profits.append(month_profit)

integrated_df = sum_specified_keep_others(dfs, sum_columns=['trade_count', 'turnover', 'commission', 'slippage', 'trading_pnl', 'holding_pnl', 'total_pnl', 'net_pnl'])
engine = BacktestingEngine()
intergrated_result = engine.calculate_statistics(integrated_df, output=False, capital=10000000)
print(f"总净利润：{sum(total_profits)}")

[datetime.datetime(2024, 4, 1, 0, 0), datetime.datetime(2024, 5, 1, 0, 0), datetime.datetime(2024, 6, 1, 0, 0), datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 8, 1, 0, 0), datetime.datetime(2024, 9, 1, 0, 0), datetime.datetime(2024, 10, 1, 0, 0), datetime.datetime(2024, 11, 1, 0, 0), datetime.datetime(2024, 12, 1, 0, 0), datetime.datetime(2025, 1, 1, 0, 0), datetime.datetime(2025, 2, 1, 0, 0), datetime.datetime(2025, 3, 1, 0, 0)]
开始日期: 2024-04-01 00:00:00
候选标的：[{'symbol': '002856', 'name': '美芝股份', 'market_value': 1165043208.0}, {'symbol': '002633', 'name': '申科股份', 'market_value': 1219500000.0}, {'symbol': '002193', 'name': '如意集团', 'market_value': 1219594472.32}, {'symbol': '002629', 'name': '仁智股份', 'market_value': 1231347360.0}, {'symbol': '001211', 'name': '双枪科技', 'market_value': 1250640000.0}, {'symbol': '002652', 'name': '扬子新材', 'market_value': 1341607680.0}, {'symbol': '000692', 'name': '惠天热电', 'market_value': 1406679014.4}, {'symbol': '000953', 'name': '河化股份', 'marke

In [ ]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.path.curdir, ".."))
sys.path.append(parent_dir)
from chart_tools import app
import dash
from dash import html, dcc, callback, Input, Output, State, ALL, MATCH
import plotly.graph_objects as go
from akshare.akshare.stock.stock_board_concept_em import stock_board_concept_hist_em
from akshare.akshare.stock.stock_zh_a_sina import stock_zh_a_daily
from akshare.akshare.index.index_stock_zh import stock_zh_index_daily_em
from plotly.subplots import make_subplots


app.run_server(debug=True, port=8883)


hover_data None
hover_data None
hover_data None
hover_data None
start date: 2024-04-01 00:00:00, end date: 2025-03-31 00:00:00, symbol: 002193.SZSE
hover_data None
hover_data None
hover_data None
hover_data None
hover_data None
start date: 2024-04-01 00:00:00, end date: 2025-03-31 00:00:00, symbol: 002193.SZSE


In [ ]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.path.curdir, ".."))
sys.path.append(parent_dir)
# from chart_tools import app
import dash
from dash import html, dcc, callback, Input, Output, State, ALL, MATCH
import plotly.graph_objects as go
from akshare.akshare.stock.stock_board_concept_em import stock_board_concept_hist_em
from akshare.akshare.stock.stock_zh_a_sina import stock_zh_a_daily
from akshare.akshare.index.index_stock_zh import stock_zh_index_daily_em
from plotly.subplots import make_subplots



concept_hist_em = stock_board_concept_hist_em('微盘股', period='daily', start_date='20240401', end_date='20250331', adjust='hfq')
hs300_index_daily = stock_zh_index_daily_em('sz399300', start_date='20240401', end_date='20250331')


integrated_df["balance"] = integrated_df["net_pnl"].cumsum()
fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
        x = integrated_df.index,
        y = integrated_df['balance'],
    ),
    secondary_y=False
)

# fig.add_trace(go.Scatter(
#     x=concept_hist_em['日期'],
#     y=concept_hist_em['收盘']
#     ),
#     secondary_y=True
# )

fig.add_trace(go.Scatter(
    x=hs300_index_daily['date'],
    y=hs300_index_daily['close']
    ),
    secondary_y=True
)

app = dash.Dash(__name__)
app.layout = html.Div([
    # html.H1("candle图", style={'textAlign': 'center'}),
    
    # 控制面板
    html.Div([
        dcc.Graph(
            id='balance-graph',
            figure=fig
        )
    ], style={'width': '200%', 'height': '100%','margin': '20px', 'display': 'inline-block'})
])
app.run_server(debug=True, port=8883)


In [5]:

from vnpy_ctabacktester.ui.widget import CandleChartDialog
from vnpy.trader.object import TradeData, BarData
from vnpy_ctastrategy.backtesting import (
    BacktestingEngine,
    OptimizationSetting,
    BacktestingMode
)
import sys
from vnpy.trader.ui import QtCore, QtWidgets, QtGui, create_qapp
def show_candle_chart(engine_ins: BacktestingEngine) -> None:
    """"""
    try:
        app = QtWidgets.QApplication.instance()
        if app is None:
            app = create_qapp('show_candle_chart')
        candle_dialog: CandleChartDialog = CandleChartDialog()
        if not candle_dialog.is_updated():
            history: list = engine_ins.history_data
            candle_dialog.update_history(history)

            trades: list[TradeData] = engine_ins.get_all_trades()
            candle_dialog.update_trades(trades)

        candle_dialog.exec_()
    except Exception as e:
        print(f"显示K线图表失败: {e}")
        return
    
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas import DataFrame, Series


def show_chart(df: DataFrame, candle_df: DataFrame) -> go.Figure:
    """显示回测结果图表"""

    fig = make_subplots(
        rows=6,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.06
    )

    balance_line = go.Scatter(
        x=df.index,
        y=df["balance"],
        mode="lines",
        name="Balance"
    )

    drawdown_scatter = go.Scatter(
        x=df.index,
        y=df["drawdown"],
        fillcolor="red",
        fill='tozeroy',
        mode="lines",
        name="Drawdown"
    )
    pnl_bar = go.Bar(y=df["net_pnl"], x=df.index, name="Daily Pnl")

    fig.add_trace(balance_line, row=1, col=1)
    fig.add_trace(drawdown_scatter, row=2, col=1)
    fig.add_trace(pnl_bar, row=3, col=1)
    fig.add_trace(go.Candlestick(
        x=candle_df.index,
        open=candle_df['open_price'],
        high=candle_df['high_price'],
        low=candle_df['low_price'],
        close=candle_df['close_price'],
        name="OHLC"
    ), row=4, col=1)
    
    colors = ['green' if close < open else 'red' 
              for close, open in zip(candle_df['close_price'], candle_df['open_price'])]
    
    fig.add_trace(
        go.Bar(
            x=candle_df.index,
            y=candle_df['volume'],
            name="成交量",
            marker_color=colors,
            opacity=0.7
        ),
        row=5, col=1
    )

    fig.update_xaxes(title_text="日期", row=5, col=1)
    fig.update_yaxes(title_text="balance", row=1, col=1)
    fig.update_yaxes(title_text="drawdown", row=2, col=1)
    fig.update_yaxes(title_text="Daily Pnl", row=3, col=1)
    fig.update_yaxes(title_text="价格", row=4, col=1)
    fig.update_yaxes(title_text="成交量", row=5, col=1)

    fig.update_layout(height=800, 
                    width=800, 
                    xaxis4_rangeslider_visible = False,
                    xaxis5_rangeslider=dict(
                        visible=True,
                        thickness=0.01,  # 减小滑块厚度
                        bgcolor="lightgray"
                    )
)
    return fig

In [ ]:
# len(record_engines)
# record_engines[0].history_data
# candle_dialog: CandleChartDialog = CandleChartDialog()
show_candle_chart(record_engines[0])


In [ ]:
from collections import defaultdict

df = list(record_df_dicts[0].values())[0]

results = defaultdict(list)
test = list(record_candle_dicts[0].values())[0]
for daily_bar in test:
    for key, value in daily_bar.__dict__.items():
        results[key].append(value)
    # results['date'] = daily_bar.datetime.strftime("%Y-%m-%d %H:%M:%S")
candle_df = DataFrame.from_dict(results).set_index('datetime')
# print(type(test))
show_chart(df, candle_df)

In [ ]:
setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("atr_length", 25, 27, 1)
setting.add_parameter("atr_ma_length", 10, 30, 10)

engine.run_ga_optimization(setting)

In [ ]:
engine.run_bf_optimization(setting)

,date,open,close,high,low,volume,amount
0,2024-04-01,3551.58,3595.65,3600.92,3551.58,162776561,2.516373e+11
1,2024-04-02,3593.22,3580.68,3596.19,3568.75,141272972,2.202075e+11
2,2024-04-03,3578.68,3567.80,3582.43,3558.87,135592331,2.142229e+11
3,2024-04-08,3553.56,3536.41,3567.82,3535.77,155304283,2.432493e+11
4,2024-04-09,3533.38,3533.49,3543.71,3518.86,129670942,1.958935e+11
...,...,...,...,...,...,...,...
236,2025-03-25,3937.90,3932.30,3946.36,3921.53,140475399,2.392463e+11
237,2025-03-26,3930.21,3919.36,3942.03,3916.86,131164187,2.209432e+11
238,2025-03-27,3911.38,3932.41,3952.99,3903.65,126793738,2.321736e+11
239,2025-03-28,3930.78,3915.17,3934.23,3907.38,126356214,2.133792e+11


,日期,开盘,收盘,最高,最低,涨跌幅,涨跌额,成交量,成交额,振幅,换手率
0,2024-04-01,1427.83,1460.56,1460.56,1427.83,2.62,37.27,25506320,1.976344e+10,2.30,3.27
1,2024-04-02,1462.32,1473.96,1479.78,1461.86,0.92,13.40,27917562,2.158200e+10,1.23,3.58
2,2024-04-03,1472.36,1462.38,1472.70,1440.89,-0.79,-11.58,28306878,2.127419e+10,2.16,3.63
3,2024-04-08,1456.58,1396.10,1456.58,1395.29,-4.53,-66.28,29838998,2.142211e+10,4.19,3.83
4,2024-04-09,1396.25,1425.95,1425.95,1396.25,2.14,29.85,21121380,1.550453e+10,2.13,2.71
...,...,...,...,...,...,...,...,...,...,...,...
236,2025-03-25,2138.46,2136.30,2150.38,2101.05,-0.22,-4.65,28397854,2.230271e+10,2.30,3.64
237,2025-03-26,2130.01,2188.62,2199.85,2126.33,2.45,52.32,28481226,2.210952e+10,3.44,3.65
238,2025-03-27,2186.46,2171.59,2188.37,2137.39,-0.78,-17.03,26832296,2.106199e+10,2.33,3.44
239,2025-03-28,2167.81,2123.31,2171.29,2123.28,-2.22,-48.28,22532262,1.795066e+10,2.21,2.89
